<a href="https://colab.research.google.com/github/sidmahurkar/Udacity_GenAI_Nanodegree/blob/main/GenAI_Nanodegree_Real_Estate_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Required Libraries

In [1]:
!pip install openai langchain pandas chromadb tik

In [2]:
!pip install langchain-community langchain-core

In [3]:
!pip install tiktoken

In [3]:
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 1.8 MB/s eta 0:00:00


In [1]:
import os

os.environ["OPENAI_API_KEY"] = "voc-1788229605126677338113966994c93dc95c1.38476665"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [57]:
#imports
import openai
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
import pandas as pd
from io import StringIO

In [38]:
#initialize the LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0.0, max_tokens = 4000)

In [39]:
#generate 25 real estate listings with the below mentioned different properties
response = llm.invoke([HumanMessage(content="""
generate 25 csv formated real estate listings. Each listing will have the following fields:
1. Neighbourhood
2. Price
3. Bedrooms
4. Bathrooms
5. House Size (in square feet)
6. Description of the property
7. Neighbourhood Description
be creative with your generated listings. csv format is a comma separated value file.
""")])

In [40]:
print(response.content)

Neighbourhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighbourhood Description
Downtown,500000,2,1,1000,"Cozy condo in the heart of the city. Perfect for young professionals or couples looking to be close to all the action.","Downtown is known for its vibrant nightlife and trendy restaurants."
Suburbia,700000,3,2,1500,"Spacious family home with a large backyard. Ideal for families looking for a quiet neighbourhood.","Suburbia is a family-friendly area with great schools and parks."
Waterfront,1000000,4,3,2000,"Luxurious waterfront property with stunning views of the ocean. Perfect for those who love to entertain.","The waterfront neighbourhood is known for its upscale dining options and beautiful beaches."
Historic District,800000,3,2,1800,"Charming historic home with original features. Ideal for those who appreciate character and craftsmanship.","The historic district is filled with beautiful architecture and rich history."
Gated Community,1200000,5,4,2500,"Elegant estate in 

In [42]:
# Convert the response content to a DataFrame
df = pd.read_csv(StringIO(response.content))

# Save the DataFrame to a CSV file
df.to_csv('real_estate_listings_25.csv', index=False)

In [43]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv('real_estate_listings_25.csv')

In [44]:
df

,Neighbourhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighbourhood Description
0,Downtown,500000,2,1,1000,Cozy condo in the heart of the city. Perfect f...,Downtown is known for its vibrant nightlife an...
1,Suburbia,700000,3,2,1500,Spacious family home with a large backyard. Id...,Suburbia is a family-friendly area with great ...
2,Waterfront,1000000,4,3,2000,Luxurious waterfront property with stunning vi...,The waterfront neighbourhood is known for its ...
3,Historic District,800000,3,2,1800,Charming historic home with original features....,The historic district is filled with beautiful...
4,Gated Community,1200000,5,4,2500,Elegant estate in a gated community with top-o...,The gated community offers exclusive access to...
5,Mountain View,600000,3,2,1600,Modern mountain retreat with panoramic views. ...,The mountain view neighbourhood is surrounded ...
6,Lakefront,900000,4,3,2100,Contemporary lakefront property with a private...,The lakefront neighbourhood is known for its p...
7,Downtown Loft,750000,2,2,1400,Stylish loft in a historic building downtown. ...,The downtown loft neighbourhood is filled with...
8,Family-Friendly,650000,4,3,1700,Spacious family home in a quiet cul-de-sac. Pe...,The family-friendly neighbourhood has a strong...
9,Luxury Condo,1100000,3,3,2200,Upscale condo with designer finishes and city ...,The luxury condo neighbourhood offers exclusiv...


In [45]:
#import required packages for vector database and generating embeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import CSVLoader
from langchain.docstore.document import Document

In [46]:
#convert listings into LangChain Document objects
documents = []
for index, row in df.iterrows():

    #create a single meaningful sentence using these variables
    listing_text = f"A {row['Bedrooms']}-bedroom, {row['Bathrooms']}-bathroom building in the {row['Neighbourhood']} neighborhood, priced at ${row['Price']} with a house size of {row['House Size']} square feet. {row['Description']} The {row['Neighbourhood Description']}"

    #ensure listing_text is a string
    listing_text = str(listing_text)

    documents.append(
        Document(
            page_content=listing_text,
            metadata={
                "Neighbourhood": row['Neighbourhood'],
                "Price": row['Price'],
                "Bedrooms": row['Bedrooms'],
                "Bathrooms": row['Bathrooms'],
                "House Size": row['House Size'],
                "Description": row['Description'],
                "Neighbourhood Description": row['Neighbourhood Description']
            }
        )
    )

In [47]:
#initialize the embedding model from LangChain
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

#store documents and embeddings in ChromaDB using LangChain
vector_store = Chroma.from_documents(documents, embedding_model)

In [48]:
#buyer preference questions and answers
questions = [
    "How big do you want your house to be?",
    "What are 3 most important things for you in choosing this property?",
    "Which amenities would you like?",
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?",
]

answers = [
    "A comfortable two-bedroom house with a spacious kitchen and a cozy living room.",
    # "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A house that is near a lake or river where with easy access to the waterbody.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

#combine answers into a single string
buyer_preferences = " ".join(answers)

#generate embedding for buyer preferences
buyer_embedding = embedding_model.embed_query(buyer_preferences)

#query ChromaDB to get top 5 matches based on buyer preferences
search_results = vector_store.similarity_search_by_vector(buyer_embedding, k=5)

#display the top 5 matches
print("Top 5 Real Estate Listings Matching Buyer Preferences:\n")
for result in search_results:
    print(result.page_content, '\n')

Top 5 Real Estate Listings Matching Buyer Preferences:

A 3-bedroom, 2-bathroom building in the Suburbia neighborhood, priced at $700000 with a house size of 1500 square feet. Spacious family home with a large backyard. Ideal for families looking for a quiet neighbourhood.. the neighbourhood Suburbia is a family-friendly area with great schools and parks. 

A 3-bedroom, 2-bathroom building in the Suburbia neighborhood, priced at $700000 with a house size of 1500 square feet. Spacious family home with a large backyard. Ideal for families looking for a quiet neighbourhood.. the neighbourhood Suburbia is a family-friendly area with great schools and parks. 

A 3-bedroom, 2-bathroom building in the Lake House neighborhood, priced at $950000 with a house size of 1900 square feet. Cozy lake house with a dock and lakeside deck. Perfect for those who love to fish and kayak.. the neighbourhood The lake house neighbourhood is known for its tranquil setting and water activities. 

A 2-bedroom, 2-

In [49]:
search_results

[Document(metadata={'Bathrooms': 2, 'Bedrooms': 3, 'Description': 'Spacious family home with a large backyard. Ideal for families looking for a quiet neighbourhood.', 'House Size': 1500, 'Neighbourhood': 'Suburbia', 'Neighbourhood Description': 'Suburbia is a family-friendly area with great schools and parks.', 'Price': 700000}, page_content='A 3-bedroom, 2-bathroom building in the Suburbia neighborhood, priced at $700000 with a house size of 1500 square feet. Spacious family home with a large backyard. Ideal for families looking for a quiet neighbourhood.. the neighbourhood Suburbia is a family-friendly area with great schools and parks.'),
 Document(metadata={'Bathrooms': 2, 'Bedrooms': 3, 'Description': 'Spacious family home with a large backyard. Ideal for families looking for a quiet neighbourhood.', 'House Size': 1500, 'Neighbourhood': 'Suburbia', 'Neighbourhood Description': 'Suburbia is a family-friendly area with great schools and parks.', 'Price': 700000}, page_content='A 3-b

In [54]:
#initialize llm for augmenting the listing description
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature = 0)

In [55]:
#import chatpromttemlate
from langchain_core.prompts import ChatPromptTemplate

In [56]:
#define function to augment description
def personalize_listing_description(listing, preferences):

    #define the prompt with a system and human message. we state that the augmenting of the listing
    #should be done in such a way that the facutal statements are not altered
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are helping a real estate agent. A potential buyer is looking for a home with the following preferences: Preferences: {preferences}
                The listing details are:
                {listing}
                """,
                ),

              (
                "human",
                "Please rewrite the listing to subtly emphasize only the features that match the buyer's preferences and keeping the factual information of listing intact."
              ),
        ]
    )

    chain = prompt | llm
    response = chain.invoke(
      {
        "preferences": preferences,
        "listing": listing
      }
    )

    return response

# Augment the top 5 retrieved listings using the buyer's preferences
print("Personalized Real Estate Listings:\n")
for result in search_results:
    personalized_listing = personalize_listing_description(result.page_content, buyer_preferences)
    print(personalized_listing.content, '\n')
    print("-----")

Personalized Real Estate Listings:

Introducing a charming 3-bedroom, 2-bathroom home nestled in the desirable Suburbia neighborhood, priced at $700,000. This spacious 1500 square feet family home boasts a large backyard, perfect for gardening enthusiasts. Ideal for those seeking a tranquil setting, this property offers a balance between suburban peace and access to urban conveniences. Located in a family-friendly area with top-rated schools and parks nearby, this home is a haven for families looking for a quiet retreat. 

-----
Introducing a charming 3-bedroom, 2-bathroom home nestled in the desirable Suburbia neighborhood, priced at $700,000. This spacious family home offers a cozy living room, a modern, energy-efficient heating system, and a large backyard perfect for gardening enthusiasts. With easy access to a nearby lake, this property provides a serene setting for water lovers. Enjoy the convenience of a two-car garage and a family-friendly community with excellent schools and p